# Create Movie Dataset

Part 1: Using the Kaggle dataset with 45K movies, query the API for all to store just the 17K movies that COMP585 catalogue contains
Since the features for each movie align exactly with the ones in the Kaggle dataset, this is likely the source of course's data.

Part 2: using a csv provided by Ben from the Azure db, preprocess that file to create a clean csv to save to the vector db. 

## Kaggle Data

In [1]:
import numpy as np
import pandas as pd
import requests

In [25]:
# Load movies_metadata.csv as it contains all relevant information to our use case
df = pd.read_csv('./kaggle_dataset/movies_metadata.csv')

/var/folders/4z/6j2481fj32g_l0n2xqz4n7j80000gn/T/ipykernel_87591/1146502047.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./kaggle_dataset/movies_metadata.csv')


In [26]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [27]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [28]:
# COMP585 movie IDs are in the format: movie+name+in+lowercase+releaseyear, 
# from the original_title column and the year portion of the release_date
# e.g. "The Dark Knight" released in 2008 would be "the+dark+knight+2008" with + as spaces
df['movie_id'] = df['original_title'].str.lower() + ' ' + df['release_date'].str[:4]
df['movie_id'] = df['movie_id'].str.replace(' ', '+')

In [29]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movie_id
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,toy+story+1995
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,jumanji+1995


In [31]:
# Query the API using the movie_id's to see which ones are in our catalogue, storing the ones that exist in a new dataframe
# As this takes time, we regularly save the results to a csv file so we can load it in the future
base_url = 'http://fall2023-comp585.cs.mcgill.ca:8080/movie/'

def movie_in_catalogue(movie_id):
    try:
        r = requests.get(base_url + movie_id)
        return r.status_code == 200
    except:
        return False

# Keep only the rows that have a movie in the catalogue
df = df[df['movie_id'].apply(movie_in_catalogue)]

# Save it
df.to_csv('./kaggle_dataset/comp585_movies.csv', index=False)

In [32]:
len(df)

17644

## Ben's Azure DB Data

In total, 6291 new movies to add from the Oct 22 dump. 

In [31]:
df = pd.read_csv('./kaggle_dataset/azure_movies_oct22.csv')

In [33]:
# Drop columns with no values for the title column
df = df.dropna(subset=['title'])

In [36]:
df.head(2)

,title,imdb_id,tmdb_id,original_title,adult,belongs_to_collection,poster_path,budget,genres,homepage,...,popularity,production_companies,production_countries,release_date,spoken_languages,status,vote_average,vote_count,movie_id,original_language
5,À la mode,tt0106878,349394.0,À la mode,False,NaN,/qbpjtFUPpYHtILQOLlgMImV1Cyo.jpg,0.0,NaN,NaN,...,0.000082,NaN,NaN,1993-09-10,NaN,Released,0.0,0.0,+la+mode+1993,en
6,À l'aventure,tt1235552,64124.0,À l'aventure,False,NaN,/stLMvwAMxd8MBnnvjI9fjiP52lb.jpg,0.0,NaN,NaN,...,3.282339,NaN,NaN,2008-08-29,NaN,Released,5.0,13.0,+laventure+2008,fr


In [37]:
# Load comp585_movies.csv, which is filtered already, and keep only movies that aren't in that dataset already
df_comp585 = pd.read_csv('./kaggle_dataset/comp585_movies.csv')

In [39]:
# Drop any rows in df that are in df_comp585, based on movie_id
df = df[~df['movie_id'].isin(df_comp585['movie_id'])]

In [41]:
len(df)

6291

In [ ]:
df.to_csv('./kaggle_dataset/azure_movies_oct22_filtered.csv', index=False)

## Create a merge of data from Kaggle and from Azure

In [2]:
df_kaggle = pd.read_csv('./kaggle_dataset/comp585_movies.csv')
df_azure = pd.read_csv('./kaggle_dataset/azure_movies_oct22_filtered.csv')

In [7]:
# Merge the two 
df = pd.concat([df_kaggle, df_azure])
# Delete any duplicates
df = df.drop_duplicates(subset=['movie_id'])

In [8]:
len(df)

23909

In [9]:
df.to_csv('./kaggle_dataset/comp585_movies_final.csv', index=False)